# NN Model III
- Scale Attack and Defense relative to enemy HP
- include Legendary field, and Level

In [10]:
import tensorflow
from tensorflow import keras
import tensorflow.keras.layers as layers

import sys
import numpy as np
import os
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

from kedro.extras.datasets.pandas import CSVDataSet

### Loading Data

In [2]:
BattleResults = catalog.load('Battle_Results_Train')

2020-05-26 13:17:22,458 - kedro.io.data_catalog - INFO - Loading data from `Battle_Results_Train` (CSVDataSet)...


In [3]:
BattleResults.head()

,Name_1,Level_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult
0,Psyduck,8,68,65,59,82,63,69,False,Weedle,75,336,192,172,103,103,279,False,Unknown,-336
1,Rapidash,86,691,707,492,571,571,752,False,Zapdos,72,607,523,499,722,523,577,True,Sunshine,133
2,Dratini,93,636,698,484,538,538,538,False,Marowak,73,446,529,720,328,529,284,False,Windy,324
3,Horsea,6,41,47,85,85,29,72,False,Moltres,86,919,886,797,1088,750,797,True,Night,-919
4,Golbat,78,571,519,449,400,481,567,False,Kingler,96,637,1178,1049,456,456,682,False,Unknown,-560


### Data Pipeline

In [16]:
Encoder = OrdinalEncoder()

In [17]:
pipeline = Pipeline([
    ('add column', addColumn()),
    ("dfdf", ColumnTransformer([
        ("Weather", Encoder, ['WeatherAndTime']),
        ("Weather2", Encoder, ['add'])
    ]))
])
pipeline.fit_transform(BattleResults)

array([[3., 3.],
       [2., 2.],
       [4., 4.],
       ...,
       [3., 3.],
       [4., 4.],
       [4., 4.]])

In [8]:
class addColumn(BaseEstimator, TransformerMixin):
    def __init__(self): # no *args or **kargs
        pass
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, df):
        df['add'] = df['WeatherAndTime']
        return df
    
class HPRelativeScaler(BaseEstimator, TransformerMixin):
    """
    Attack_x relative to HP_y
    Defense_x relative to HP_x
    
    """
    def __init__(self): # no *args or **kargs
        pass
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        for i in [1, 2]:
            X[f'Attack_{i}'] /= X[f'HP_{3-i}']
            X[f'Sp_Atk_{i}'] /= X[f'HP_{3-i}']
            X[f'Defense_{i}'] /= X[f'HP_{i}']
            X[f'Sp_Def_{i}'] /= X[f'HP_{i}']
        X.drop(columns=['HP_1', 'HP_2'])
        return X.values

In [9]:
numerical_attributes = ['HP_1', 'Attack_1', 'Defense_1', 'Sp_Atk_1', 'Sp_Def_1', 'Speed_1', 
                          'HP_2', 'Attack_2', 'Defense_2', 'Sp_Atk_2', 'Sp_Def_2', 'Speed_2']
full_pipeline = ColumnTransformer([
        ("Numerical Attributes", StandardScaler(), numerical_attributes),
        ("Boolean", "passthrough", ['Legendary_1', 'Legendary_2']),
        ("Level" , LevelScaler(), ['Level_1', 'Level_2']),
    ])
full_pipeline.fit(BattleResults); #battle_data

2020-05-25 08:23:19,001 - numexpr.utils - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2020-05-25 08:23:19,001 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [49]:
x_train = full_pipeline.transform(BattleResults_train)
x_val = full_pipeline.transform(BattleResults_val)

In [50]:
x_train

array([[ 0.10318571, -0.314413  , -0.35908019, ...,  0.        ,
         0.71717172,  0.78787879],
       [ 1.30515212,  0.23830986,  1.21471438, ...,  0.        ,
         0.86868687,  0.64646465],
       [-0.64852027,  0.45515605, -0.04255309, ...,  0.        ,
         0.47474747,  0.1010101 ],
       ...,
       [ 1.19449489,  0.41910742, -1.12569525, ...,  0.        ,
         0.64646465,  0.09090909],
       [-0.3051013 , -0.35946723, -0.94433481, ...,  0.        ,
         0.62626263,  0.45454545],
       [-0.7591775 , -0.27638261, -0.57727756, ...,  0.        ,
         0.08080808,  0.2020202 ]])

### Model

- With Level and and Legendary 0.12

In [51]:
# 10 epochs, Validation  RMSE: 0.17
model = keras.models.Sequential([
    layers.Input(shape=(x_train.shape[1],)),
    layers.Dense(200, activation='relu'),
    layers.Dense(200, activation='relu'),
    layers.Dense(200, activation='relu'),
    layers.Dense(1, activation='tanh')
])

In [112]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=3e-3,
    decay_steps=10000,
    decay_rate=0.9)

In [52]:
model.compile(loss='mse',
              optimizer=keras.optimizers.RMSprop(learning_rate=3e-3),
              metrics=['RootMeanSquaredError'])

### Train

In [53]:
history = model.fit(x_train, y_train, epochs=20,
                    validation_data=(x_val, y_val),
                    batch_size=1024)

Epoch 1/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0826 - root_mean_squared_error: 0.2873 - val_loss: 0.0710 - val_root_mean_squared_error: 0.2664
Epoch 2/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0672 - root_mean_squared_error: 0.2592 - val_loss: 0.0610 - val_root_mean_squared_error: 0.2469
Epoch 3/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0581 - root_mean_squared_error: 0.2410 - val_loss: 0.0523 - val_root_mean_squared_error: 0.2287
Epoch 4/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0503 - root_mean_squared_error: 0.2242 - val_loss: 0.0482 - val_root_mean_squared_error: 0.2194
Epoch 5/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0439 - root_mean_squared_error: 0.2096 - val_loss: 0.0415 - val_root_mean_squared_error: 0.2037
Epoch 6/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0390 - root_mean_squared_error: 0.1975 - val_l

In [54]:
history = model.fit(x_train, y_train, epochs=20,
                    validation_data=(x_val, y_val),
                    batch_size=1024)

Epoch 1/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0185 - root_mean_squared_error: 0.1359 - val_loss: 0.0216 - val_root_mean_squared_error: 0.1470
Epoch 2/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0182 - root_mean_squared_error: 0.1348 - val_loss: 0.0218 - val_root_mean_squared_error: 0.1475
Epoch 3/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0178 - root_mean_squared_error: 0.1335 - val_loss: 0.0193 - val_root_mean_squared_error: 0.1390
Epoch 4/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0176 - root_mean_squared_error: 0.1326 - val_loss: 0.0181 - val_root_mean_squared_error: 0.1344
Epoch 5/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0173 - root_mean_squared_error: 0.1314 - val_loss: 0.0203 - val_root_mean_squared_error: 0.1425
Epoch 6/20
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0171 - root_mean_squared_error: 0.1307 - val_l